In [2]:
import MySQLdb
import pandas
import csv
import sys

In [3]:
proj_ids = []

with open('ids.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count > 0:
            proj_ids.append(row[0])
        line_count += 1



In [4]:
projects = pandas.DataFrame(columns=("ghId", "slug", "prTimeDiff", "prCount"))

db = MySQLdb.connect(passwd="ghtorrent_restore",db="ghtorrent_06_19",user="ghtorrent")

c = db.cursor()

done = 0

for id in proj_ids:

    c.execute("""
        select pr.id as gh_id, pr.name as slug, TIMESTAMPDIFF(MONTH, min(iss.created_at), max(iss.created_at)) as timediff, count(iss.id) as prs 
        from projects pr, issues iss
        where iss.pull_request = 1 and iss.repo_id = {} and pr.id = {}
        group by pr.id
        having TIMESTAMPDIFF(MONTH, min(iss.created_at), max(iss.created_at)) > 24 and count(iss.id) > 24;""".format(id))

    rowCount = 0
    
    row = c.fetchone()

    if row is not None:
        projects.loc[rowCount] = [row[0], row[1], row[2], row[3]]
        
    done += 1

    sys.stdout.write("\r{0}".format((float(done)/len(proj_ids))*100))
    sys.stdout.flush()

0.008369563718330808

KeyboardInterrupt: 

In [ ]:
print(projects.summary())